<a href="https://colab.research.google.com/github/ashev2021/Fine-Tuning-LLM/blob/main/Copy_of_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "datasets>=2.18.0,<3"
# deploy in Gradio
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

In [ ]:
!pip install transformers>=4.38.2 sentence-transformers>=2.5.1 setfit>=1.0.3 accelerate>=0.27.2 seqeval>=1.2.2

In [ ]:
#Load dataset
from datasets import load_dataset

# Prepare data and splits
imdb_data = load_dataset("mteb/imdb")

train_data, test_data = imdb_data["train"], imdb_data["test"]


train_data = train_data.shuffle(seed=42).select(range(2000))
test_data = test_data.shuffle(seed=42).select(range(1000))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
#Supervised classification Fine tune everything(Huggingface train)

# model and tokenize
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"

#pretrained model + tokenizer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)



config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Data Collator
from transformers import DataCollatorWithPadding

# pad to longest sentence

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# preprocess
def preprocess_function(examples):
   """Tokenize input data"""
   return tokenizer(examples["text"], truncation=True)

# tokenize train
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# evaluate the model with F1
import numpy as np
import evaluate


def compute_metrics(eval_pred):
    """Calculate F1 score"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    load_f1 = evaluate.load("f1")
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"f1": f1}


In [ ]:
# training arguments for parameter tuning
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=1,
   weight_decay=0.01,
   save_strategy="epoch",
   report_to="none"
)

# Trainer which executes the training process
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)



/tmp/ipython-input-7-4173415651.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# train
trainer.train()

Step,Training Loss


TrainOutput(global_step=125, training_loss=0.46621987915039065, metrics={'train_runtime': 95.5269, 'train_samples_per_second': 20.937, 'train_steps_per_second': 1.309, 'total_flos': 261395434004160.0, 'train_loss': 0.46621987915039065, 'epoch': 1.0})

In [ ]:
#eval results
trainer.evaluate()

{'eval_loss': 0.32598134875297546,
 'eval_f1': 0.8654618473895582,
 'eval_runtime': 14.8586,
 'eval_samples_per_second': 67.301,
 'eval_steps_per_second': 4.24,
 'epoch': 1.0}

In [ ]:
#save fine-tuned model
trainer.save_model("my_finetuned_model")


In [ ]:
import gradio as gr
from transformers import pipeline

# Load your fine-tuned model
fine_tuned_model_path = "my_finetuned_model"
fine_tuned_classifier = pipeline("text-classification", model=fine_tuned_model_path, tokenizer=fine_tuned_model_path)

# Load pretrained sentiment model from Hugging Face Hub
pretrained_classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased")

# Label mapping
label_map = {
    "LABEL_0": "negative",
    "LABEL_1": "positive",
    "NEGATIVE": "negative",
    "POSITIVE": "positive",
}

def compare_models(text):
    # Fine-tuned model
    ft_pred = fine_tuned_classifier(text)[0]
    ft_label = label_map.get(ft_pred["label"], ft_pred["label"])
    ft_score = round(ft_pred["score"], 4)
    ft_result = f"Label: {ft_label}, Confidence: {ft_score}"

    # Pretrained model
    pre_pred = pretrained_classifier(text)[0]
    pre_label = label_map.get(pre_pred["label"], pre_pred["label"])
    pre_score = round(pre_pred["score"], 4)
    pre_result = f"Label: {pre_label}, Confidence: {pre_score}"

    return ft_result, pre_result

with gr.Blocks() as demo:
    gr.Markdown("## 🎬 Compare Fine-Tuned vs Pretrained Sentiment Models")
    gr.Markdown("Enter a movie review below to compare predictions.")

    input_text = gr.Textbox(label="Movie Review", lines=4, placeholder="Type your review here...")
    classify_btn = gr.Button("Classify")

    with gr.Row():
        ft_output = gr.Textbox(label="Fine-Tuned Model Prediction", interactive=False)
        pre_output = gr.Textbox(label="Pretrained Model Prediction", interactive=False)

    classify_btn.click(compare_models, inputs=input_text, outputs=[ft_output, pre_output])

demo.launch(share=True)
